# KubeVirt

KubeVirt ist ein Open-Source-Projekt, das es ermöglicht, virtuelle Maschinen auf Kubernetes bereitzustellen, zu nutzen und zu verwalten – ähnlich wie Container. 

Durch eine einheitliche Bereitstellungs- und Verwaltungsebene für sowohl Container als auch virtuelle Maschinen entsteht eine vereinheitlichte Plattform für cloud-native Anwendungen, unabhängig von den Anforderungen.

Quelle: Unterhaltung mit Copilot, 18.8.2024
* (1) Getting Started with KubeVirt Containers and Virtual Machines Together. https://www.redhat.com/en/blog/getting-started-with-kubevirt.
* (2) Introduction | KubeVirt.io. https://kubevirt.io/docs/.
* (3) GitHub - kubevirt/kubevirt: Kubernetes Virtualization API and runtime .... https://github.com/kubevirt/kubevirt.
* (4) Overview of KubeVirt | Kube by Example. https://kubebyexample.com/learning-paths/kubevirt-fundamentals/overview-kubevirt.

- - -

Reset Kubernetes (microk8s)

In [11]:
%%bash
sudo snap remove microk8s
# neu Installation
curl -sfL https://raw.githubusercontent.com/mc-b/lerncloud/main/services/microk8s.sh | bash -
curl -sfL https://raw.githubusercontent.com/mc-b/lerncloud/main/services/microk8saddons.sh | bash -

microk8s removed
microk8s (1.31/stable) v1.31.3 from Canonical** installed


snap "kubectl" is already installed, see 'snap help refresh'
snap "helm" is already installed, see 'snap help refresh'
Infer repository core for addon dns


Enabling DNS
Using host configuration from /run/systemd/resolve/resolv.conf
Applying manifest
serviceaccount/coredns created
configmap/coredns created
deployment.apps/coredns created
service/kube-dns created
clusterrole.rbac.authorization.k8s.io/coredns created
clusterrolebinding.rbac.authorization.k8s.io/coredns created
CoreDNS service deployed with IP address 10.152.183.10
DNS is enabled
poddisruptionbudget.policy/calico-kube-controllers configured
serviceaccount/calico-kube-controllers unchanged
serviceaccount/calico-node unchanged
configmap/calico-config unchanged
customresourcedefinition.apiextensions.k8s.io/bgpconfigurations.crd.projectcalico.org configured
customresourcedefinition.apiextensions.k8s.io/bgppeers.crd.projectcalico.org configured
customresourcedefinition.apiextensions.k8s.io/blockaffinities.crd.projectcalico.org configured
customresourcedefinition.apiextensions.k8s.io/caliconodestatuses.crd.projectcalico.org configured
customresourcedefinition.apiextensions.k8s.io/c


What is MicroK8s?
- - -

Weitere Informationen: [https://microk8s.io/](https://microk8s.io/)
  


Infer repository core for addon ingress
Infer repository core for addon metrics-server


         This form of chained operations on addons will be DEPRECATED in the future.
         Please, enable one addon at a time: 'microk8s enable <addon>'
Enabling Ingress
ingressclass.networking.k8s.io/public created
ingressclass.networking.k8s.io/nginx created
namespace/ingress created
serviceaccount/nginx-ingress-microk8s-serviceaccount created
clusterrole.rbac.authorization.k8s.io/nginx-ingress-microk8s-clusterrole created
role.rbac.authorization.k8s.io/nginx-ingress-microk8s-role created
clusterrolebinding.rbac.authorization.k8s.io/nginx-ingress-microk8s created
rolebinding.rbac.authorization.k8s.io/nginx-ingress-microk8s created
configmap/nginx-load-balancer-microk8s-conf created
configmap/nginx-ingress-tcp-microk8s-conf created
configmap/nginx-ingress-udp-microk8s-conf created
daemonset.apps/nginx-ingress-microk8s-controller created
Ingress is enabled
Enabling Metrics-Server
serviceaccount/metrics-server created
clusterrole.rbac.authorization.k8s.io/system:aggregated-metrics-re

In [2]:
! sudo microk8s enable storage

Infer repository core for addon storage
DEPRECATION WARNING: 'storage' is deprecated and will soon be removed. Please use 'hostpath-storage' instead.

Infer repository core for addon hostpath-storage
Enabling default storage class.
         A hostpath volume can grow beyond the size limit set in the volume claim manifest.

deployment.apps/hostpath-provisioner created
storageclass.storage.k8s.io/microk8s-hostpath created
serviceaccount/microk8s-hostpath created
clusterrole.rbac.authorization.k8s.io/microk8s-hostpath created
clusterrolebinding.rbac.authorization.k8s.io/microk8s-hostpath created
Storage will be available soon.


In [ ]:
%%bash
# Funktion zum Hinzufügen der Option --allow-privileged=true zu einer Datei
add_allow_privileged() {
    local file=$1
    if ! grep -q -- "--allow-privileged=true" "$file"; then
        echo "--allow-privileged=true" | sudo tee -a "$file"
        echo "Option --allow-privileged=true zu $file hinzugefügt."
    else
        echo "Option --allow-privileged=true ist bereits in $file vorhanden."
    fi
}

# Pfade zu den Konfigurationsdateien
kubelet_config="/var/snap/microk8s/current/args/kubelet"
kube_apiserver_config="/var/snap/microk8s/current/args/kube-apiserver"

# Option zu den Konfigurationsdateien hinzufügen
add_allow_privileged "$kubelet_config"
add_allow_privileged "$kube_apiserver_config"

microk8s stop
microk8s start

### Validate Hardware Virtualization Support
Hardware with virtualization support is recommended. You can use virt-host-validate to ensure that your hosts are capable of running virtualization workloads:

In [ ]:
! sudo usermod -aG kvm $USER
! sudo chmod 666 /dev/kvm


In [3]:
! virt-host-validate qemu

  QEMU: Checking for hardware virtualization                                 : PASS
  QEMU: Checking if device /dev/kvm exists                                   : PASS
  QEMU: Checking if device /dev/kvm is accessible                            : PASS
  QEMU: Checking if device /dev/vhost-net exists                             : PASS
  QEMU: Checking if device /dev/net/tun exists                               : PASS
  QEMU: Checking for cgroup 'cpu' controller support                         : PASS
  QEMU: Checking for cgroup 'cpuacct' controller support                     : PASS
  QEMU: Checking for cgroup 'cpuset' controller support                      : PASS
  QEMU: Checking for cgroup 'memory' controller support                      : PASS
  QEMU: Checking for cgroup 'devices' controller support                     : WARN (Enable 'devices' in kernel Kconfig file or mount/enable cgroup controller in your system)
  QEMU: Checking for cgroup 'blkio' controller support               

Und Installieren KubeVirt mittels Operators

In [6]:
%%bash
export VERSION=$(curl -s https://storage.googleapis.com/kubevirt-prow/release/kubevirt/kubevirt/stable.txt)
echo $VERSION

kubectl apply -f https://github.com/kubevirt/kubevirt/releases/download/${VERSION}/kubevirt-operator.yaml
kubectl apply -f https://github.com/kubevirt/kubevirt/releases/download/${VERSION}/kubevirt-cr.yaml

# kubectl -n kubevirt wait kv kubevirt --for condition=Available

# enabling nested virtualization
kubectl -n kubevirt patch kubevirt kubevirt --type=merge --patch '{"spec":{"configuration":{"developerConfiguration":{"useEmulation":true}}}}'

v1.4.0
namespace/kubevirt unchanged
customresourcedefinition.apiextensions.k8s.io/kubevirts.kubevirt.io unchanged
priorityclass.scheduling.k8s.io/kubevirt-cluster-critical configured
clusterrole.rbac.authorization.k8s.io/kubevirt.io:operator unchanged
serviceaccount/kubevirt-operator unchanged
role.rbac.authorization.k8s.io/kubevirt-operator unchanged
rolebinding.rbac.authorization.k8s.io/kubevirt-operator-rolebinding unchanged
clusterrole.rbac.authorization.k8s.io/kubevirt-operator unchanged
clusterrolebinding.rbac.authorization.k8s.io/kubevirt-operator unchanged
deployment.apps/virt-operator configured
kubevirt.kubevirt.io/kubevirt configured
kubevirt.kubevirt.io/kubevirt patched


Kontrollieren, bzw. warten bis alles läuft.

Es sollten 6 Pods laufen

In [10]:
! kubectl get kubevirt.kubevirt.io/kubevirt -n kubevirt -o=jsonpath="{.status.phase}"
! kubectl get pods -n kubevirt

DeployingNAME                                                              READY   STATUS    RESTARTS   AGE
kubevirt-181122bb566b78f3958f071886d59e622871f649-jobk9nmcttq7b   0/1     Pending   0          5m46s
virt-operator-79456d8689-ntj2f                                    1/1     Running   0          6m11s
virt-operator-79456d8689-xxjnz                                    1/1     Running   0          6m11s


In [ ]:
! kubectl logs -n kubevirt kubevirt-181122bb566b78f3958f071886d59e622871f649-job46m4nd8c4d 

Wir brauchen zusätzlich das CLI `virtctl` um VMs zu starten, stoppen oder sich mit diesen zu verbinden:

In [ ]:
%%bash
export VERSION=$(curl -s https://api.github.com/repos/kubevirt/kubevirt/releases/latest | jq -r .tag_name)
echo ${VERSION}
wget https://github.com/kubevirt/kubevirt/releases/download/$VERSION/virtctl-$VERSION-linux-amd64
chmod +x virtctl-$VERSION-linux-amd64
sudo mv virtctl-$VERSION-linux-amd64 /usr/local/bin/virtctl
virtctl

- - -

### Erstellen von VMs

Zuerst brauchen wir eine Deklaration im YAML Format

In [ ]:
%%bash
cat <<%EOF% | kubectl apply -f - 
apiVersion: kubevirt.io/v1
kind: VirtualMachine
metadata:
  name: ubuntu-vm
  namespace: default
spec:
  running: false
  template:
    metadata:
      labels:
        kubevirt.io/domain: ubuntu-vm
    spec:
      domain:
        cpu:
          cores: 2
        devices:
          disks:
          - disk:
              bus: virtio
            name: containerdisk
          - disk:
              bus: virtio
            name: cloudinitdisk
          interfaces:  # Netzwerkschnittstellen hier definieren
          - name: default
            masquerade: {}  # Verwende das Masquerade-Netzwerk für NAT
        memory:
          guest: 4Gi
      networks:
      - name: default
        pod: {}  # Verweist auf das Standard-Pod-Netzwerk
      volumes:
      - name: containerdisk
        containerDisk:
          image: quay.io/containerdisks/ubuntu:24.04
      - name: cloudinitdisk
        cloudInitNoCloud:
          userData: |
            #cloud-config
            users:
              - name: ubuntu
                passwd: insecure
                lock_passwd: false
                sudo: ALL=(ALL) NOPASSWD:ALL
                ssh_pwauth: true
            ssh_pwauth: true
%EOF%


In [ ]:
! virtctl start ubuntu-vm

In [ ]:
! kubectl get vm

- - -

### Aufräumen

In [ ]:
! kind delete cluster 
! kubectl config use-context microk8s